In [ ]:
import numpy as np
import sys
#np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4
from helper_PFCI import PFHamiltonianGenerator
from helper_PFCI import Determinant
from helper_cqed_rhf import cqed_rhf
np.set_printoptions(threshold=sys.maxsize)
psi4.core.set_output_file('output.dat', False)

In [ ]:
# options for mgf
mol_str = """
Li
H 1 1.5
symmetry c1
"""

options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
    "save_jk" : True
}


psi4.set_options(options_dict)
mol = psi4.geometry(mol_str)


In [ ]:
mol_str = """
Li
H 1 1.5
symmetry c1
"""

options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}

cavity_dict = {
    'omega_value' : 0.0,
    'lambda_vector' : np.array([0, 0, 0]),
    'ci_level' : 'fci',
    'full_diagonalization' : True,
    'no_cavity' : True
}

mol = psi4.geometry(mol_str)

psi4.set_options(options_dict)

test_pf = PFHamiltonianGenerator(
    mol_str,
    options_dict,
    cavity_dict
)



In [ ]:
def reduce_ci_space(instance):
    """
    function to map QED-CI space to electronic subspace for when 
    there is no cavity effect
    """
    # make the CI matrix in the electronic subspace only
    _H_CI_NO_CAV = instance.ApDmatrix + instance.Enuc_so + instance.dc_so
    
    # diagonize only the electronic subspace CI matrix
    instance.CIeigs, instance.CIvecs = np.linalg.eigh(_H_CI_NO_CAV)
    
    # get the number of determinants
    _numDets = len(instance.CIeigs)
    
    # reduce the dipole arrays to the electronic subspace
    instance.MU_X = np.copy(instance.MU_X[:_numDets, :_numDets])
    instance.MU_Y = np.copy(instance.MU_Y[:_numDets, :_numDets])
    instance.MU_Z = np.copy(instance.MU_Z[:_numDets, :_numDets])


This block will identify $N_{el}$ of the excited-states with optically-allowed transitions from the ground-state.

In [ ]:
def sort_dipole_allowed_states(instance, N_el):
    """
    A function to capture the indices of states with a dipole-allowed
    singlet transition from the ground-state
    """
    
    # we want to find the first N_el states that are coupled to the ground state through a dipole transition
    _singlet_states = [0]
    _num_kets = 0
    _ket_idx = 1
    _sing_idx = 1
    zero_vec = np.array([0., 0., 0.])
    while _num_kets < N_el - 1:
        _tmp_mu = instance.compute_dipole_moment(0, _ket_idx)
        if np.allclose(zero_vec, _tmp_mu):
            _ket_idx += 1
        else:
            _singlet_states.append(_ket_idx)
            _ket_idx += 1
            _num_kets += 1
            _sing_idx += 1
            
    return _singlet_states


def compute_dipole_moments(instance, states):
    """
    Given an array of states, compute all the dipole moments between those states
    """
    _Ns = len(states)
    _singlet_dipole_moments = np.zeros((_Ns, _Ns, 3))
    
    for i in range(_Ns):
        a = states[i]
        for j in range(_Ns):
            b = states[j]
            _singlet_dipole_moments[i, j, :] = instance.compute_dipole_moment(a, b) + instance.mu_nuc * (a==b)
    
    return _singlet_dipole_moments




In [ ]:
N_el = 20
reduce_ci_space(test_pf)
singlet_states = sort_dipole_allowed_states(test_pf, N_el)
print(singlet_states)


In [ ]:
print(compute_dipole_moments(test_pf, singlet_states))

In [ ]:
mol_tmpl = """
Li
H 1 **R**
symmetry c1
"""

mol_str = """
Li
H 1 1.5
symmetry c1
"""

N_R = 50
N_el = 20
r_array = np.linspace(1.4, 2.2, N_R)

E_array = np.zeros((N_R, N_el))
Mu_array = np.zeros((N_R, N_el, N_el, 3))
psi4.set_options(options_dict)

r_idx = 0
for r in r_array:
    mol_str = mol_tmpl.replace("**R**", str(r))
    print(mol_str)
    mol = psi4.geometry(mol_str)
    #psi4.set_options({'restricted_docc': [3],'active': [4],'num_roots':2})
    scf_e, wfn = psi4.energy('SCF', return_wfn=True)
    fci_energy = psi4.energy('fci',ref_wfn=wfn)
    
    LiH = PFHamiltonianGenerator(mol_str, options_dict, cavity_dict)
    assert np.isclose(fci_energy, LiH.CIeigs[0])
    reduce_ci_space(LiH)
    singlet_states = sort_dipole_allowed_states(LiH,  N_el)
    E_array[r_idx, :] = LiH.CIeigs[singlet_states]
    Mu_array[r_idx, :, :, :] = compute_dipole_moments(LiH, singlet_states)
    r_idx += 1
    
    
    #CASCI_E_array.append(H2_PF.CIeigs[0])
    #FCI_E_array.append(fci_energy)
    #print(r, H2_PF.CIeigs[0])


In [ ]:
from matplotlib import pyplot as plt
print(r_array)
print(E_array[:,1]-E_array[:,0])

In [ ]:
plt.plot(r_array, E_array[:,0]+0.120000)
plt.plot(r_array, E_array[:,1])
plt.plot(r_array, E_array[:,2])

plt.show()

\begin{equation}
\hat{H}_{\alpha n, \beta m} = \left( E_{\alpha}(R) + n \omega \right) \delta_{\alpha \beta} \delta_{nm} + \sqrt{\frac{\omega}{2}} \lambda \cdot \mu_{\alpha \beta}(R) \left(\sqrt{n} \delta_{n,m-1} + \sqrt{n+1} \delta_{n,m+1} \right)  \\
+\frac{1}{2} \sum_{\gamma = 1}^N \lambda \cdot \mu_{\alpha, \gamma}(R) \lambda \cdot \mu_{\gamma, \beta} \delta_{nm}
\end{equation}


In [ ]:
def build_pf_hamiltonian(n_el, n_ph, E_R, omega, lamvec, mu):
    """
    Given an array of n_el E_R values and an n_ph states with fundamental energy omega
    build the PF Hamiltonian
    
    n_el : int
        the number of electronic states (n_el = 1 means only ground-state)
    
    n_ph : int
        the number of photon occupation states (n_ph = 1 means only the |0> state)
    
    E_R : np.array of floats
        the electronic energies
        
    omega : float
        the energy of the photonic mode
        
    lamvec : np.array of floats
        the lambda vector
        
    mu : (n_el x n_el x 3) np.array of floats 
        mu[i, j, k] is the kth cartesian component of the dipole moment expectation value between 
        state i and state j
    
    """
    H_PF = np.zeros((n_el * n_ph, n_el * n_ph))
    
    # take care of the diagonals first
    # bare electronic and photonic energy
    for n in range(n_ph):
        for a in range(n_el):
            na = n * n_el + a
            H_PF[na,na] = E_R[a] + n * omega
        
    # diagonal dipole self energy
    for n in range(n_ph):
        for a in range(n_el):
            na = n * n_el + a
            for g in range(n_el):
                H_PF[na,na] += 0.5 * np.dot(lamvec, mu[a,g,:]) * np.dot(lamvec, mu[g,a,:])
            
    # off-diagonal dipole self energy
    for n in range(n_ph):
        for a in range(n_el):
            na = n * n_el + a
            for b in range(n_el):
                nb = n * n_el + b
                for g in range(n_el):
                    if a != b:
                        H_PF[na, nb] += 0.5 * np.dot(lamvec, mu[a,g,:]) * np.dot(lamvec, mu[g, b, :])
                
    # off-diagonal bilinear coupling
    for n in range(n_ph):
        for a in range(n_el):
            na = n * n_el + a
            
            for m in range(n_ph):
                for b in range(n_el):
                    mb = m * n_el + b
                    
                    if n == (m-1) and a != b:
                        #print(n, a, na, m, b, mb)
                        H_PF[na,mb] = -np.sqrt(omega / 2) * np.dot(lamvec, mu[a,b,:]) * np.sqrt(m) 
                        H_PF[mb, na] = -np.sqrt(omega / 2) * np.dot(lamvec, mu[a,b,:]) * np.sqrt(m) 
                        
                    elif n == (m+1) and a != b:
                        #print(n, a, na, m, b, mb)
                        H_PF[na, mb] = -np.sqrt(omega / 2) * np.dot(lamvec, mu[a,b,:]) * np.sqrt(m+1) 
                        H_PF[mb, na] = -np.sqrt(omega / 2) * np.dot(lamvec, mu[a,b,:]) * np.sqrt(m+1)
                        
    
    return H_PF
                
            
    
        
    
    

\begin{equation}
\hat{H}_{\alpha n, \beta m} = \left( E_{\alpha}(R) + n \omega \right) \delta_{\alpha \beta} \delta_{nm} + \sqrt{\frac{\omega}{2}} \lambda \cdot \mu_{\alpha \beta}(R) \left(\sqrt{n} \delta_{n,m-1} + \sqrt{n+1} \delta_{n,m+1} \right)  \\
+\frac{1}{2} \sum_{\gamma = 1}^N \lambda \cdot \mu_{\alpha, \gamma}(R) \lambda \cdot \mu_{\gamma, \beta} \delta_{nm}
\end{equation}


In [ ]:
np.set_printoptions(precision=6, linewidth=200, suppress=True)
omega_cav = 0.120000
lambda_vector = np.array([0, 0.0, 0.05])


# Just manually computing some elements!
#print("E")
#print(E_array[0,:2], E_array[0,:2]+0.12, E_array[0,:2]+2*0.12)

#g_12 = np.sqrt(omega_cav / 2) * np.dot(lambda_vector, Mu_array[0, 0, 1, :]) * np.sqrt(2)
#D_01 = 0.5 * np.dot(lambda_vector, Mu_array[0, 0, 0, :]) * np.dot(lambda_vector, Mu_array[0, 1, 0, :]) 
#D_01 += 0.5 * np.dot(lambda_vector, Mu_array[0, 0, 1, :]) * np.dot(lambda_vector, Mu_array[0, 1, 1, :])
#print("g12 * 1")
#print(g_12)
#print("D01")
#print(D_01)
#+E_array[0,1])

_HPF = build_pf_hamiltonian(N_el, N_ph, E_array[0,:], omega_cav, lambda_vector, Mu_array[0, :, :, :])
print("H")
print(_HPF)

In [ ]:
N_el = 20
N_ph = 20

PF_EIG = np.zeros((N_R, N_el * N_ph))

In [ ]:

for i in range(N_R):
    _HPF = build_pf_hamiltonian(N_el, N_ph, E_array[i,:], omega_cav, lambda_vector, Mu_array[i, :, :, :])
    pf_e, pf_c = np.linalg.eigh(_HPF)
    PF_EIG[i,:] = pf_e
    print(r_array[i], pf_e[0], pf_e[1],  pf_e[2], pf_e[2])

In [ ]:
#plt.plot(r_array, PF_EIG[:,0])
#plt.plot(r_array, E_array[:,1])
plt.plot(r_array, PF_EIG[:,1], 'r--')
plt.plot(r_array, PF_EIG[:,2], 'b--')
#plt.plot(r_array, E_array[:,0]+omega_cav)
#plt.plot(r_array, E_array[:,2])
plt.plot(r_array, E_array[:,3])
#plt.plot(r_array, PF_EIG[:,3])
#plt.plot(r_array, PF_EIG[:,4])
#plt.plot(r_array, PF_EIG[:,5])





In [ ]:
pf_e, pf_c = np.linalg.eigh(_HPF)
print(pf_e)

In [ ]:
print(pf_e[2]-pf_e[0])
#print(pf_e[3])
print(omega_cav)
#print(pf_e[1]-pf_e[0])
#print(pf_e[2]-pf_e[1])

In [ ]:
expected_mghp_g_e = -199.86358254419457
expected_mghp_lp_e = -199.69776087489558
expected_mghp_up_e = -199.68066502792058

print(expected_mghp_lp_e-expected_mghp_g_e)
print(expected_mghp_up_e-expected_mghp_lp_e)